In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
train = pd.read_csv('https://raw.githubusercontent.com/Nikhil-V98/Analyticsvidhya/main/Black_friday/train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/Nikhil-V98/Analyticsvidhya/main/Black_friday/test.csv')

In [ ]:
print("train :" ,train.shape)
print("test :" ,test.shape)

train : (550068, 12)
test : (233599, 11)


In [ ]:
train.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN,1422
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,NaN,1057
4,1000002,P00285442,M,55+,16,C,4+,0,8,NaN,NaN,7969


In [ ]:
test.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3
0,1000004,P00128942,M,46-50,7,B,2,1,1,11.0,NaN
1,1000009,P00113442,M,26-35,17,C,0,0,3,5.0,NaN
2,1000010,P00288442,F,36-45,1,B,4+,1,5,14.0,NaN
3,1000010,P00145342,F,36-45,1,B,4+,1,4,9.0,NaN
4,1000011,P00053842,F,26-35,1,C,1,0,4,5.0,12.0


In [8]:
((train.isnull().sum())/train.shape[0])*100

User_ID                        0.000000
Product_ID                     0.000000
Gender                         0.000000
Age                            0.000000
Occupation                     0.000000
City_Category                  0.000000
Stay_In_Current_City_Years     0.000000
Marital_Status                 0.000000
Product_Category_1             0.000000
Product_Category_2            31.566643
Product_Category_3            69.672659
Purchase                       0.000000
dtype: float64

In [9]:
((test.isnull().sum())/test.shape[0])*100

User_ID                        0.000000
Product_ID                     0.000000
Gender                         0.000000
Age                            0.000000
Occupation                     0.000000
City_Category                  0.000000
Stay_In_Current_City_Years     0.000000
Marital_Status                 0.000000
Product_Category_1             0.000000
Product_Category_2            30.969311
Product_Category_3            69.590195
dtype: float64

In [12]:
test.fillna(0,inplace=True)
train.fillna(0,inplace=True)

In [16]:
train.describe()

,User_ID,Occupation,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
count,5.500680e+05,550068.000000,550068.000000,550068.000000,550068.000000,550068.000000,550068.000000
mean,1.003029e+06,8.076707,0.409653,5.404270,6.735436,3.841941,9263.968713
std,1.727592e+03,6.522660,0.491770,3.936211,6.215492,6.250712,5023.065394
min,1.000001e+06,0.000000,0.000000,1.000000,0.000000,0.000000,12.000000
25%,1.001516e+06,2.000000,0.000000,1.000000,0.000000,0.000000,5823.000000
50%,1.003077e+06,7.000000,0.000000,5.000000,5.000000,0.000000,8047.000000
75%,1.004478e+06,14.000000,1.000000,8.000000,14.000000,8.000000,12054.000000
max,1.006040e+06,20.000000,1.000000,20.000000,18.000000,18.000000,23961.000000


In [21]:
col_name = train.columns
col_name

Index(['User_ID', 'Product_ID', 'Gender', 'Age', 'Occupation', 'City_Category',
       'Stay_In_Current_City_Years', 'Marital_Status', 'Product_Category_1',
       'Product_Category_2', 'Product_Category_3', 'Purchase'],
      dtype='object')

In [30]:
train.dtypes

User_ID                         int64
Product_ID                     object
Gender                         object
Age                            object
Occupation                      int64
City_Category                  object
Stay_In_Current_City_Years     object
Marital_Status                  int64
Product_Category_1              int64
Product_Category_2            float64
Product_Category_3            float64
Purchase                        int64
dtype: object

In [44]:
for features in col_name:
    print(features,'-->',len(train[features].unique()),':', train[features].unique())

User_ID --> 5891 : [1000001 1000002 1000003 ... 1004113 1005391 1001529]
Product_ID --> 3631 : ['P00069042' 'P00248942' 'P00087842' ... 'P00370293' 'P00371644'
 'P00370853']
Gender --> 2 : ['F' 'M']
Age --> 7 : ['0-17' '55+' '26-35' '46-50' '51-55' '36-45' '18-25']
Occupation --> 21 : [10 16 15  7 20  9  1 12 17  0  3  4 11  8 19  2 18  5 14 13  6]
City_Category --> 3 : ['A' 'C' 'B']
Stay_In_Current_City_Years --> 5 : ['2' '4+' '3' '1' '0']
Marital_Status --> 2 : [0 1]
Product_Category_1 --> 20 : [ 3  1 12  8  5  4  2  6 14 11 13 15  7 16 18 10 17  9 20 19]
Product_Category_2 --> 18 : [ 0.  6. 14.  2.  8. 15. 16. 11.  5.  3.  4. 12.  9. 10. 17. 13.  7. 18.]
Product_Category_3 --> 16 : [ 0. 14. 17.  5.  4. 16. 15.  8.  9. 13.  6. 12.  3. 18. 11. 10.]
Purchase --> 18105 : [ 8370 15200  1422 ...   135   123   613]


In [47]:
gender_dict = {'F':0, 'M':1}
age_dict = {'0-17':17, '18-25':25, '26-35':35, '36-45':45, '46-50':50, '51-55':55, '55+':60}
city_dict = {'A':0, 'B':1, 'C':2}
stay_dict = {'0':0, '1':1, '2':2, '3':3, '4+':4}

train['Gender'] = train['Gender'].map(gender_dict)
test['Gender'] = test['Gender'].map(gender_dict)

train['Age'] = train['Age'].map(age_dict)
test['Age'] = test['Age'].map(age_dict)

train['City_Category'] = train['City_Category'].map(city_dict)
test['City_Category'] = test['City_Category'].map(city_dict)

train['Stay_In_Current_City_Years'] = train['Stay_In_Current_City_Years'].map(stay_dict)
test['Stay_In_Current_City_Years'] = test['Stay_In_Current_City_Years'].map(stay_dict)


In [50]:
for features in col_name:
    print(features,'-->',len(train[features].unique()),':', train[features].unique())

User_ID --> 5891 : [1000001 1000002 1000003 ... 1004113 1005391 1001529]
Product_ID --> 3631 : ['P00069042' 'P00248942' 'P00087842' ... 'P00370293' 'P00371644'
 'P00370853']
Gender --> 2 : [0 1]
Age --> 7 : [17 60 35 50 55 45 25]
Occupation --> 21 : [10 16 15  7 20  9  1 12 17  0  3  4 11  8 19  2 18  5 14 13  6]
City_Category --> 3 : [0 2 1]
Stay_In_Current_City_Years --> 5 : [2 4 3 1 0]
Marital_Status --> 2 : [0 1]
Product_Category_1 --> 20 : [ 3  1 12  8  5  4  2  6 14 11 13 15  7 16 18 10 17  9 20 19]
Product_Category_2 --> 18 : [ 0.  6. 14.  2.  8. 15. 16. 11.  5.  3.  4. 12.  9. 10. 17. 13.  7. 18.]
Product_Category_3 --> 16 : [ 0. 14. 17.  5.  4. 16. 15.  8.  9. 13.  6. 12.  3. 18. 11. 10.]
Purchase --> 18105 : [ 8370 15200  1422 ...   135   123   613]


In [51]:
train.dtypes

User_ID                         int64
Product_ID                     object
Gender                          int64
Age                             int64
Occupation                      int64
City_Category                   int64
Stay_In_Current_City_Years      int64
Marital_Status                  int64
Product_Category_1              int64
Product_Category_2            float64
Product_Category_3            float64
Purchase                        int64
dtype: object

In [59]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()
scaler = StandardScaler()

train['User_ID']=enc.fit_transform(train['User_ID'],)
test['User_ID']=enc.transform(test['User_ID'])

In [64]:
train.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,0,672,0,17,10,0,2,0,3,0.0,0.0,8370
1,0,2376,0,17,10,0,2,0,1,6.0,14.0,15200
2,0,852,0,17,10,0,2,0,12,0.0,0.0,1422
3,0,828,0,17,10,0,2,0,12,14.0,0.0,1057
4,1,2734,1,60,16,2,4,0,8,0.0,0.0,7969


In [62]:
train['Product_ID'].replace('P00','')

0          672
1         2376
2          852
3          828
4         2734
          ... 
550063    3567
550064    3568
550065    3568
550066    3568
550067    3566
Name: Product_ID, Length: 550068, dtype: int32

In [60]:
train['Product_ID'] = train['Product_ID'].str.replace('P00', '')
test['Product_ID'] = test['Product_ID'].str.replace('P00', '')

train['Product_ID'] = scaler.fit_transform(train['Product_ID'].values.reshape(-1, 1))
test['Product_ID'] = scaler.transform(test['Product_ID'].values.reshape(-1, 1))

AttributeError: Can only use .str accessor with string values!